# Task 2

In this notebook, we prepare and train a variational circuit that transforms four 4-qubit input states $(|\phi_1\rangle, |\phi_2\rangle, |\phi_3\rangle, |\phi_4\rangle)$ into predefined output states:

$$
|\phi_1\rangle \rightarrow |0011\rangle  \\
|\phi_2\rangle \rightarrow |0101\rangle \\
|\phi_3\rangle \rightarrow |1010\rangle  \\
|\phi_4\rangle \rightarrow |1100\rangle \\
$$

For the variational circuit, we use an alternating operator ansatz, with the odd layers composed of $U3$ gates applied to each qubit and the even layers composed of $XX(\varphi)$ gates applied pairwise.

In [ ]:
import pennylane as qml
from pennylane import numpy as np

In [ ]:
num_qubits = 4

dev1 = qml.device("default.qubit", wires=num_qubits)

In [ ]:
def odd_layer(params):
    qml.U3(params[0:3], wires=0)
    qml.U3(params[3:6], wires=1)
    qml.U3(params[6:9], wires=2)
    qml.U3(params[9:12], wires=3)

In [ ]:
def even_layer(params):
    for q1 in range(4):
        for q2 in range(q1,4):
            qml.IsingXX(params[q1+q2], wires=[q1,q2])

In [ ]:
def layer(params):
    even_layer(params[0:6])
    odd_layer(params[6:18])

In [ ]:
def init_input(state_idx):
    return [
        qml.QubitStateVector([1,0]+[0 for i in range(2**3)], wires=[0,1,2,3]),
        qml.QubitStateVector([0 for i in range(2**3)]+[0,1], wires=[0,1,2,3]),
        qml.QubitStateVector([0,1]+[0 for i in range(2**3)], wires=[0,1,2,3]),
        qml.QubitStateVector([0 for i in range(2**3)]+[1,0], wires=[0,1,2,3]),
    ][state_idx]

In [ ]:
n_layers = 3

state_idx = 0 # change this to change input state

params = np.random.normal(0, np.pi, n_layers*18)

@qml.qnode(dev1)
def circuit(params):
    init_input(state_idx)
    for layer in range(n_layers):
        layer(params[18*layer:18*(layer+1)])
    return qml.state()

In order to train the circuit, we have to define a cost function. The simplest is to use the overlap of the final state of the circuit with the target output state

In [ ]:
output_states = [
    np.array([0 for i in range(7)]+[1]+[0 for i in range(8)]),
    np.array([0 for i in range(9)]+[1]+[0 for i in range(6)]),
    np.array([0 for i in range(10)]+[1]+[0 for i in range(5)]),
    np.array([0 for i in range(12)]+[1]+[0 for i in range(3)])
]

In [ ]:
def cost_fn(params, state_idx):
    c = circuit(params)
    return np.dot(c, np.conj(output_states[state_idx]))

In [ ]:
# set up the optimizer
opt = qml.AdamOptimizer()

steps = 200

best_cost = cost_fn(params, state_idx)
best_params = np.zeros(18*n_layers)

print("Cost after 0 steps is {:.4f}".format(cost_fn(params)))

# optimization begins
for n in range(steps):
    params = opt.step(cost_fn, params)
    current_cost = cost_fn(params)

    # keeps track of best parameters
    if current_cost < best_cost:
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, current_cost))
